In [1]:
import argparse
import logging
import numpy as np
import os
import sys
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.init import xavier_uniform_
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from utils import parsing
from utils.data_utils import PolymerDataset
from utils.train_utils import get_lr, grad_norm, NoamLR, param_count, param_norm, set_seed, setup_logger, load_data, batch_force_reshape, MTAdam
from models.polygnn import polygnn
import pdb
from torch.autograd import grad
from torch.utils.tensorboard import SummaryWriter

In [21]:
model_path = "/home/chenlidong/polyAttn/checkpoints/copolymer_4w_.polygnn_single_ea_no_ho/model.3000_2.pt"
model_info = torch.load(model_path)
args = model_info['args']
device = "cuda"
hps = {
        "ffn_capacity" : 2,
        "depth":3,
        "readout_dim": 128,
        "activation": nn.functional.leaky_relu
}
model = polygnn(133,147,hps)
model.load_state_dict(model_info['state_dict'])
model.to(device)
model.eval()

polygnn(
  (mpnn): polygnn_mp(
    (R): hidden_xavier_bn(
      (linear): Linear(in_features=133, out_features=128, bias=True)
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (E): ffn(
      (layers): ModuleList(
        (0): hidden_xavier_bn(
          (linear): Linear(in_features=147, out_features=128, bias=True)
          (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): hidden_xavier_bn(
          (linear): Linear(in_features=128, out_features=256, bias=True)
          (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): hidden_xavier_bn(
          (linear): Linear(in_features=256, out_features=147, bias=True)
          (bn): BatchNorm1d(147, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (mp_layer): polygnn_mp_block()
  )
  (output): output(
    (linear): Linear(in_fe

In [34]:
test_dataset = PolymerDataset(f"/home/chenlidong/polyAttn/preprocessed/{args.data_name}",phase="test")
test_loader = DataLoader(
            dataset=test_dataset,
            batch_size=args.train_batch_size,
            shuffle=True,
            pin_memory=True
        )

In [35]:
count = test_dataset.len() // 64 - 1
loss = 0.0
loss_fn = nn.functional.mse_loss
for test_idx, test_batch in enumerate(test_loader):
    if test_idx >= count:
        break
    test_batch = test_batch.to(device)
    tgt_ea = test_batch['ea']
    tgt_ip = test_batch['ip']
    
    pred_ea = model(test_batch)

    test_loss = torch.sqrt(loss_fn(pred_ea,tgt_ea))
    # print(test_loss)
    loss += test_loss.item() / count

In [36]:
loss

0.49268231781801813

In [6]:
import math

def calculate_average(numbers):
    total = sum(numbers)
    average = total / len(numbers)
    return average

def calculate_standard_deviation(numbers):
    average = calculate_average(numbers)
    variance = sum((x - average) ** 2 for x in numbers) / len(numbers)
    std_deviation = math.sqrt(variance)
    return std_deviation

In [25]:
numbers = [0.24603574224432606,0.24605163662953497,0.24612404114769817,0.24577337380190534,0.24621309194349714]

In [37]:
numbers = [0.493141615077069,0.4929233733424566,0.49268231781801813]

In [38]:
calculate_average(numbers)

0.4929157687458479

In [39]:
calculate_standard_deviation(numbers)

0.00018758440848516586